In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow as tf
from tensorflow.keras import utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2
import sys
import pandas as pd
from random import shuffle

In [3]:
 
def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

  def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
def image_sequence(directory):
    """this function takes a list of images and returns the list in bytes"""
    image_bytes_list = []
    for image in sorted((get_unify_frames(directory))):
        img = open(directory+"/"+image, 'rb').read()
        if img is None:
            continue
        image_bytes = tf.train.Feature(bytes_list=tf.train.BytesList(value=[img]))
        image_bytes_list.append(image_bytes)

    return image_bytes_list


In [4]:
def createDataRecord(out_dirname, addrs, labels, imsize, dataset):
  with tf.device('/device:GPU:0'):
    
    image_height = imsize[0]
    image_width = imsize[1]
    image_depth = imsize[2]
    
    writer = tf.io.TFRecordWriter(dataset+'.tfrecords')

    
    for i in range(len(addrs)):
        # open the TFRecords file
        # print how many images are saved every 1000 images
        if not i % 1000:
          print(str(i) +","+ str(len(addrs)))
          sys.stdout.flush()
        
        image_bytes_list = image_sequence(addrs[i])            # Load the image
        label_int_list  = [_float_feature(labels[i])]*len(image_bytes_list)
        images = tf.train.FeatureList(feature=image_bytes_list)
        labels_dat = tf.train.FeatureList(feature=label_int_list)

        im_length = tf.train.Feature(int64_list=tf.train.Int64List(value=[len(image_bytes_list)]))
        im_height = tf.train.Feature(int64_list=tf.train.Int64List(value=[image_height]))
        im_width = tf.train.Feature(int64_list=tf.train.Int64List(value=[image_width]))
        im_depth = tf.train.Feature(int64_list=tf.train.Int64List(value=[image_depth]))
        
        # Create a feature
        feature = {
        'Images': images,
        'Labels': labels_dat
        }
        
        context_dict = {'length': im_length, 'height': im_height, 'width': im_width, 'depth': im_depth}

        sequence_context = tf.train.Features(feature=context_dict)

        
        
        # Create an example protocol buffer
        example = tf.train.SequenceExample(context=sequence_context, feature_lists=tf.train.FeatureLists(feature_list=feature))
        # Serialize to string and write on the file
        writer.write(example.SerializeToString())
          
    writer.close()
    sys.stdout.flush()


In [5]:
'''
The videos do not have the same number of frames, here we try to unify.
'''
hm_frames = 30 # number of frames
# unify number of frames for each training
def get_unify_frames(path):
    offset = 0
    # pick frames
    frames = os.listdir(path)
    frames_count = len(frames)
    # unify number of frames 
    if hm_frames > frames_count:
        # duplicate last frame if video is shorter than necessary
        frames += [frames[-1]] * (hm_frames - frames_count)
    elif hm_frames < frames_count:
        # If there are more frames, then sample starting offset
        #diff = (frames_count - hm_frames)
        #offset = diff-1 
        frames = frames[0:hm_frames]
    return frames




In [6]:
def createLabels():
  labels = []
  imagesPath = '/kaggle/input/gesturevid/20bn50_part1'
  dirs = os.listdir(imagesPath)
  # Finally, classes label you want to use all labels 
  label = pd.read_csv('/kaggle/input/gesture-vid-label1/jester-v1-labels.csv',header=None, usecols=[0])
  label.head()
  targets_name = label[0].tolist()
  # training targets stored
  targets_temp = pd.read_csv('/kaggle/input/gesturevid-label/jester-v1-train.csv',header=None,sep = ";").to_dict()
  targets = {}
  for index in range(len(targets_temp[0])):
    targets[targets_temp[0][index]] = targets_temp[1][index]

  for dir in dirs:
    labels.append(targets_name.index(targets[int(dir)]))
    #todo: del targets{} if not required later
  return labels
# read addresses and labels from the 'train' folder
images_train_path = '/kaggle/input/gesturevid/20bn50_part1/*'
addrs = glob.glob(images_train_path)
labels = createLabels()
# Divide the data into 60% train, 20% validation, and 20% test
# Better to use sklearn train-test split. 


In [7]:

train_addrs = addrs[0:int(0.6*len(addrs))]
train_labels = labels[0:int(0.6*len(labels))]
val_addrs = addrs[int(0.6*len(addrs)):int(0.8*len(addrs))]
val_labels = labels[int(0.6*len(addrs)):int(0.8*len(addrs))]
test_addrs = addrs[int(0.8*len(addrs)):]
test_labels = labels[int(0.8*len(labels)):]

!mkdir /kaggle/working/train_rec/
!mkdir /kaggle/working/val_rec/
!mkdir /kaggle/working/test_rec/

createDataRecord('/kaggle/working/train_rec/', train_addrs, train_labels, (64,64,1), 'train')
createDataRecord('/kaggle/working/val_rec/', val_addrs, val_labels, (64,64,1), 'val')
createDataRecord('/kaggle/working/test_rec/', test_addrs, test_labels, (64,64,1), 'test')

0,35568
1000,35568
2000,35568
3000,35568
4000,35568
5000,35568
6000,35568
7000,35568
8000,35568
9000,35568
10000,35568
11000,35568
12000,35568
13000,35568
14000,35568
15000,35568
16000,35568
17000,35568
18000,35568
19000,35568
20000,35568
21000,35568
22000,35568
23000,35568
24000,35568
25000,35568
26000,35568
27000,35568
28000,35568
29000,35568
30000,35568
31000,35568
32000,35568
33000,35568
34000,35568
35000,35568
0,11856
1000,11856
2000,11856
3000,11856
4000,11856
5000,11856
6000,11856
7000,11856
8000,11856
9000,11856
10000,11856
11000,11856
0,11857
1000,11857
2000,11857
3000,11857
4000,11857
5000,11857
6000,11857
7000,11857
8000,11857
9000,11857
10000,11857
11000,11857
